In [85]:
from ast import literal_eval
import numpy, torch, torch.nn
from utils import *
torch.set_default_tensor_type(torch.DoubleTensor)
torch.get_default_dtype()

torch.float64

In [88]:
def load_model(filename, dtype="float64"):
    with open(filename, "r") as f:
        data = literal_eval(f.read())
    x = numpy.array(data[0], dtype=dtype)
    return x[:3], x[3:], list(data[1].keys()), numpy.array(list(data[1].values()), dtype=dtype)

def write_model(p1, p2, k, t, filename, dtype="float64"):
    with open(filename, "w") as f:
        f.write(repr([p1.tolist() + p2.tolist(), dict(zip(k, t))]))

In [89]:
write_model(numpy.array([0.3,0.2,0.5]), numpy.array([0.1,0.2,0.3, 0.05,0.1,0.25], dtype=float), [(0,1,0),(0,0,1)], numpy.array([0.5, 0.5]), "test.tree.learned")
load_model("test.tree.learned")

(array([0.3, 0.2, 0.5]),
 array([0.1 , 0.2 , 0.3 , 0.05, 0.1 , 0.25]),
 [(0, 1, 0), (0, 0, 1)],
 array([0.5, 0.5]))

In [127]:
p1 = torch.zeros(3)
p2 = torch.zeros(6)
kk = [(0,1,0), (0,0,1)]
t = torch.zeros(len(kk))

def P1(p1, N):
    M = torch.zeros((N+1,)*3)
    M[0,0,0] = 1
    pp = torch.nn.functional.softmax(p1, dim=0)
    M[1,0,0] = pp[0]
    M[0,1,0] = pp[1]
    M[0,0,1] = pp[2]
    for n in range(2, N+1):
        for i in range(0, n+1):
            for j in range(0, n+1-i):
                k = n-i-j
                if i > 0:
                    M[i,j,k] += M[i-1,j,k]*pp[0]
                if j > 0:
                    M[i,j,k] += M[i,j-1,k]*pp[1]
                if k > 0:
                    M[i,j,k] += M[i,j,k-1]*pp[2]
    return M

def P2(p2, N):
    M = torch.zeros((N+1,)*3)
    M[0,0,0] = 1
    pp = torch.nn.functional.softmax(p2, dim=0)
    # p200, p201, p220, p211, p212, p222
    # x^2    xy    xz    y^2   yz   z^2
    M[2,0,0] = pp[0]
    M[1,1,0] = pp[1]
    M[1,0,1] = pp[2]
    M[0,2,0] = pp[3]
    M[0,1,1] = pp[4]
    M[0,0,2] = pp[5]
    for n in range(4, N+1, 2):
        for i in range(0, n+1):
            for j in range(0, n+1-i):
                k = n-i-j
                # {0, 0, 2}, {0, 1, 1}, {0, 2, 0}, {1, 0, 1}, {1, 1, 0}, {2, 0, 0}
                if i > 1:
                    M[i,j,k] += M[i-2,j,k]*pp[0]
                if i > 0 and j > 0:
                    M[i,j,k] += M[i-1,j-1,k]*pp[1]
                if i > 0 and k > 0:
                    M[i,j,k] += M[i-1,j,k-1]*pp[2]
                if j > 1:
                    M[i,j,k] += M[i,j-2,k]*pp[3]
                if j > 0 and k > 0:
                    M[i,j,k] += M[i,j-1,k-1]*pp[4]
                if k > 1:
                    M[i,j,k] += M[i,j,k-2]*pp[5]
    return M

def fk(k, n, P1, P2):
    nnz = n > 0
    kappa = sum(k)
    result = torch.scalar_tensor(0)
    if n[0] - n[2] == kappa and n[0] >= k[0] and n[1] >= k[1] and n[2] >= k[2]:
        for i10 in range(max(k[0]-n[0], -n[1]), 1):
            for i11 in range(max(k[1], -i10+1), min(n[1], n[2]-k[2]-i10 + 1)):
                result += numpy.linalg.det(numpy.array([[n[0],0,0],[i10, i11, -i10-i11],[-i10+k[0]-n[0], -i11+k1, i10+i11+k[2]]])[nnz][:, nnz])* \
                             P1[-i10, n[1] - i11, i10 + i11]*P2[i10 - k[0] + n[0], i11 - k[1], n[2] - (i10 + i11 + k[2])]
    return result/n[nnz].prod()

def f(k, n, P1, P2):
    kappa = sum(k)
    result = torch.scalar_tensor(0)
    for n0 in range(kappa, (kappa+n)//2 + 1):
        result += fk(k, numpy.array([n0, kappa + n - 2*n0, n0 - kappa]), P1, P2)
    return result

In [134]:
print(P1(p1,4))
print(P2(p2,4))
f((1,0,0), 1, P1(p1,3), P2(p2,3))

tensor([[[1.0000, 0.3333, 0.1111, 0.0370, 0.0123],
         [0.3333, 0.2222, 0.1111, 0.0494, 0.0000],
         [0.1111, 0.1111, 0.0741, 0.0000, 0.0000],
         [0.0370, 0.0494, 0.0000, 0.0000, 0.0000],
         [0.0123, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.3333, 0.2222, 0.1111, 0.0494, 0.0000],
         [0.2222, 0.2222, 0.1481, 0.0000, 0.0000],
         [0.1111, 0.1481, 0.0000, 0.0000, 0.0000],
         [0.0494, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.1111, 0.1111, 0.0741, 0.0000, 0.0000],
         [0.1111, 0.1481, 0.0000, 0.0000, 0.0000],
         [0.0741, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0370, 0.0494, 0.0000, 0.0000, 0.0000],
         [0.0494, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000,

tensor(0.)

In [35]:
P1(torch.from_numpy(numpy.log(numpy.array([0.3,0.2,0.5]))), 4)

tensor([[[1.0000, 0.5000, 0.2500, 0.1250, 0.0625],
         [0.2000, 0.2000, 0.1500, 0.1000, 0.0000],
         [0.0400, 0.0600, 0.0600, 0.0000, 0.0000],
         [0.0080, 0.0160, 0.0000, 0.0000, 0.0000],
         [0.0016, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.3000, 0.3000, 0.2250, 0.1500, 0.0000],
         [0.1200, 0.1800, 0.1800, 0.0000, 0.0000],
         [0.0360, 0.0720, 0.0000, 0.0000, 0.0000],
         [0.0096, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0900, 0.1350, 0.1350, 0.0000, 0.0000],
         [0.0540, 0.1080, 0.0000, 0.0000, 0.0000],
         [0.0216, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0270, 0.0540, 0.0000, 0.0000, 0.0000],
         [0.0216, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000,

In [36]:
numpy.array([0.1,0.2,0.3, 0.05,0.1,0.25]).sum()

1.0

In [67]:
print(P2(torch.from_numpy(numpy.log(numpy.array([0.1,0.2,0.3, 0.05,0.1,0.25], dtype=float))), 6))

tensor([[[1.0000e+00, 0.0000e+00, 2.5000e-01, 0.0000e+00, 6.2500e-02,
          0.0000e+00, 1.5625e-02],
         [0.0000e+00, 1.0000e-01, 0.0000e+00, 5.0000e-02, 0.0000e+00,
          1.8750e-02, 0.0000e+00],
         [5.0000e-02, 0.0000e+00, 3.5000e-02, 0.0000e+00, 1.6875e-02,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 1.0000e-02, 0.0000e+00, 8.5000e-03, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.5000e-03, 0.0000e+00, 3.3750e-03, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 7.5000e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.2500e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 3.0000e-01, 0.0000e+00, 1.5000e-01, 0.0000e+00,
          5.6250e-02, 0.0000e+00],
         [2.0000e-01, 0.0000e+00, 1.6000e-01, 0.0000e+00, 8.2500e-02,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 7.0000e-02, 0.0000e+00, 6.1500e

In [102]:
torch.scalar_tensor(0)

tensor(0.)

In [103]:
_.dtype

torch.float64

In [109]:
fk((0,1,0), numpy.array([2,1,4]), P1, P2)

8

In [122]:
numpy.linalg.det(numpy.array([[1,0,0],[0,0,0],[0,0,1]], dtype="int32")[[0,2]][:, [0,2]])

1.0

In [123]:
numpy.array([[1,0,0],[0,0,0],[0,0,1]]).dtype

dtype('int32')